In [1]:
import json
import pandas as pd
from tqdm import tqdm

In [3]:
raw_files = "Video_Games.jsonl"
meta_files = "meta_Video_Games.jsonl"
threshold = 4

user_ids2items = {}
item_id2count = {}

for line in tqdm(open(raw_files, "r"), total=4600000):
    data = json.loads(line)
    if data["user_id"] not in user_ids2items:
        user_ids2items[data["user_id"]] = []
    if data["asin"] not in item_id2count:
        item_id2count[data["asin"]] = 0
    if data["rating"] >= threshold:
        user_ids2items[data["user_id"]].append((data["asin"], data["timestamp"]))
        item_id2count[data["asin"]] += 1

print(len(user_ids2items))
print(len(item_id2count))


  0%|          | 0/4800000 [00:00<?, ?it/s]

 96%|█████████▋| 4624615/4800000 [00:24<00:00, 189871.99it/s]

2766656
167645


In [ ]:
have_feature_items = set()
with open(meta_files, "r", encoding="utf-8") as f:
    for line in f:
        data = json.loads(line)
        if "asin" in data:
            have_feature_items.add(data["asin"])
        if "parent_asin" in data:
            have_feature_items.add(data["parent_asin"])
print(len(have_feature_items))

In [11]:
for i in range(10):
    tmp_user_dict = {}
    tmp_item_count = {}
    over5count_items = set()
    for item_id, count in item_id2count.items():
        if count >= 5 and item_id in have_feature_items:
            over5count_items.add(item_id)

    for user_id, items in user_ids2items.items():
        tmp_items = []
        for item, timestamp in items:
            if item in over5count_items:
                tmp_items.append((item, timestamp))
        if len(tmp_items) >= 5:
            tmp_user_dict[user_id] = tmp_items
            for item, timestamp in tmp_items:
                if item not in tmp_item_count:
                    tmp_item_count[item] = 0
                tmp_item_count[item] += 1
            # over5count_user.add(user_id)
    item_id2count = tmp_item_count
    user_ids2items = tmp_user_dict
    print("len(tmp_user_dict):", len(tmp_user_dict))
    print("len(tmp_item_count):", len(tmp_item_count))

len(tmp_user_dict): 19978
len(tmp_item_count): 12872
len(tmp_user_dict): 17042
len(tmp_item_count): 8768
len(tmp_user_dict): 16243
len(tmp_item_count): 7909
len(tmp_user_dict): 15943
len(tmp_item_count): 7608
len(tmp_user_dict): 15768
len(tmp_item_count): 7432
len(tmp_user_dict): 15658
len(tmp_item_count): 7322
len(tmp_user_dict): 15618
len(tmp_item_count): 7272
len(tmp_user_dict): 15600
len(tmp_item_count): 7253
len(tmp_user_dict): 15593
len(tmp_item_count): 7243
len(tmp_user_dict): 15582
len(tmp_item_count): 7233


In [5]:
for k,v in user_ids2items.items():
    user_ids2items[k] = sorted(v, key=lambda x:x[1])

In [12]:
item_havefeatures = set()
for line in tqdm(open(meta_files, "r"), total=137269):
    data = json.loads(line)
    if "asin" in data:
        print(data["asin"])
    if data["parent_asin"] not in item_id2count:
        continue
    item_havefeatures.add(data["parent_asin"])

100%|██████████| 137269/137269 [00:01<00:00, 83592.92it/s]


In [13]:
userraw_id2model_id, itemraw_id2model_id = {}, {}
user_idx, item_idx = 1, 1
not_have_feature_items, not_have_feature_count = set(), 0
interaction_file = open("VedioGames.csv", 'w', encoding='utf-8')
for user_id, items in user_ids2items.items():
    if user_id not in userraw_id2model_id:
        userraw_id2model_id[user_id] = user_idx
        user_idx += 1
    tmp_str = str(userraw_id2model_id[user_id]) + "\t"
    for item, timestamp in items:
        if item not in item_havefeatures:
            not_have_feature_items.add(item)
            not_have_feature_count += 1
            continue
        if item not in itemraw_id2model_id:
            itemraw_id2model_id[item] = item_idx
            item_idx += 1
        tmp_str += str(itemraw_id2model_id[item]) + " "
    tmp_str = tmp_str[:-1] + "\n"
    interaction_file.write(tmp_str)
interaction_file.close()

print(not_have_feature_count)
print(not_have_feature_items)
print(len(not_have_feature_items))
with open("user_id2model_id.json", "w", encoding="utf-8") as f:
    json.dump(userraw_id2model_id, f)
with open("item_id2model_id.json", "w", encoding="utf-8") as f:
    json.dump(itemraw_id2model_id, f)

0
set()
0


In [18]:
# item_id2count
feature_keys = ["title", "main_category", "categories","description", "features"]
CDsID, rawID, title, main_category, categories, description, features = [], [], [], [], [], [], []
for line in tqdm(open(meta_files, "r"), total=137269):
    data = json.loads(line)
    if data["parent_asin"] not in item_id2count:
        continue
    rawID.append(data["parent_asin"])
    CDsID.append(itemraw_id2model_id[data["parent_asin"]])
    title.append(data["title"])
    main_category.append(data["main_category"])
    categories.append(",".join(data["categories"]))
    description.append(data["description"][0] if len(data["description"]) > 0 else "")
    features.append(",".join(data["features"]))

100%|██████████| 137269/137269 [00:01<00:00, 90107.91it/s]


In [19]:
item_featuers = pd.DataFrame({"GamesID": CDsID, "rawID": rawID, "title": title, "main_category": main_category, "categories": categories, "description": description, "features": features})
item_featuers.to_csv("VedioGames_features.csv", index=False, sep="\t")